In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some fixed parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 2, 3])              # Boundary condition function coefficients
dim_z = 30                            # Latent space dimension
dtype = torch.float                  # Tensor data type

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def essBoundaryFun(x):
    return 0.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [5]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.setNaturalRHS(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace, ksp)
rhs.setRhsStencil(mesh, K)

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs)

In [7]:
trainingData = dta.StokesData(range(256))
trainingData.readData(['IMG'])
# trainingData.plotMicrostruct(1)
trainingData.reshapeInputImg()

In [8]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [24]:
steps = int(50)
for s in range(steps):
    print('step = ', s)
    batchSamples_pf = torch.multinomial(torch.ones(trainingData.nSamplesIn), model.batchSizeN_pf)
#     batchSamples_pc = torch.multinomial(torch.ones(trainingData.nSamplesOut), model.batchSizeN_pc)
    model.optLatentDistStep()
    model.pfStep(batchSamples_pf)
#     model.pcStep(batchSamples_pc)
    

step =  0
loss_pf =  tensor(3346.8145, grad_fn=<SubBackward0>)
step =  1
loss_pf =  tensor(3338.9922, grad_fn=<SubBackward0>)
step =  2
loss_pf =  tensor(3331.2234, grad_fn=<SubBackward0>)
step =  3
loss_pf =  tensor(3323.5146, grad_fn=<SubBackward0>)
step =  4
loss_pf =  tensor(3315.8589, grad_fn=<SubBackward0>)
step =  5
loss_pf =  tensor(3308.2622, grad_fn=<SubBackward0>)
step =  6
loss_pf =  tensor(3300.7168, grad_fn=<SubBackward0>)
step =  7
loss_pf =  tensor(3293.2251, grad_fn=<SubBackward0>)
step =  8
loss_pf =  tensor(3285.7888, grad_fn=<SubBackward0>)
step =  9
loss_pf =  tensor(3278.3989, grad_fn=<SubBackward0>)
step =  10
loss_pf =  tensor(3271.0630, grad_fn=<SubBackward0>)
step =  11
loss_pf =  tensor(3263.7769, grad_fn=<SubBackward0>)
step =  12
loss_pf =  tensor(3256.5439, grad_fn=<SubBackward0>)
step =  13
loss_pf =  tensor(3249.3586, grad_fn=<SubBackward0>)
step =  14
loss_pf =  tensor(3242.2271, grad_fn=<SubBackward0>)
step =  15
loss_pf =  tensor(3235.1431, grad_fn=<S

In [27]:
model.plotInputReconstruction()
f = plt.gcf()
f.suptitle('Untrained, N = 1184', fontsize=32, y=.7)

Text(0.5, 0.7, 'Untrained, N = 1184')

In [26]:
f.suptitle('N = 256', fontsize=32, y=.7)

Text(0.5, 0.7, 'N = 256')

In [12]:
model.z_mean

tensor([[ -2.2437,   8.7265,  -5.8289,  ...,   6.3580,   1.9284,   4.2061],
        [  3.0961,   1.2469,   0.3156,  ...,   1.5052,  -3.7843,  -2.6156],
        [  4.8751,   3.7017,  -0.9524,  ...,   6.5499,   1.3157,  -9.9536],
        ...,
        [ -2.4814,  -5.0901,  -0.2269,  ...,  -3.6274,   0.5880,  -2.2263],
        [  3.1659,  -8.4665,  -2.6156,  ...,   7.3822,   8.8450,  -0.4655],
        [  9.6971, -10.2827,  -0.8787,  ...,  -4.2116,   0.7959,  -1.9728]],
       requires_grad=True)

In [29]:
sample = model.pfNet(model.z_mean[0, :])

In [30]:
sample.shape

torch.Size([65536])

In [31]:
plt.figure()

<Figure size 640x480 with 0 Axes>

In [32]:
plt.imshow(torch.reshape(sample, (model.data.imgResolution, model.data.imgResolution)).detach().numpy() > .5,
                     cmap='binary')

In [35]:
plt.figure()
plt.imshow(torch.reshape(sample, (model.data.imgResolution, model.data.imgResolution)).detach().numpy())

In [36]:
plt.figure()
trainingData.plotMicrostruct(0)